In [1]:
import numpy as np
import pandas as pd
import toytree
from scipy.optimize import minimize
from scipy.linalg import expm

In [4]:
def datatodict(data):
    """
    Parses data into format that can be used by the cond_like and
    pruningalg functions
    """
    values = [{0:-(i-1),1:i} for i in data]
    keys = list(range(0, len(mydata), 1))
    valuesdict = dict(zip(keys,values))
    return valuesdict

In [5]:
def cond_like(likeleft0, likeleft1, likeright0, likeright1, tL, tR, alpha, beta):
    """
    Calculates conditional likelihood of character states at each node
    """

    Q = np.array([[-alpha, alpha], [beta, -beta]])
    probleft = expm(Q*tL)
    probright = expm(Q*tR)
 
    #ancestor is 0
    left0 = probleft[0, 0] * likeleft0 + probleft[0, 1] * likeleft1
    right0 = probright[0, 0] * likeright0 + probright[0, 1] * likeright1
    like_zero = left0*right0
 
    #ancestor is 1
    left1 = probleft[1, 0] * likeleft0 + probleft[1, 1] * likeleft1
    right1 = probright[1, 0] * likeright0 + probright[1, 1] * likeright1
    like_one = left1*right1
 
    return {0: like_zero, 1: like_one}

In [6]:
def pruningalg(tree, alpha, beta):
    """
    Runs Felsenstein's pruning algorithm on an input tree, given instantaneous transition
    rates alpha and beta. Assigns likelihood scores for characters states at each node.
    Specifically for binary state model. 
    """
    for node in tree.treenode.traverse("postorder"):
        if len(node.children) == 2:
            child1 = node.children[0]
            child2 = node.children[1]
            likedict = cond_like(likeright0 = child1.likelihood[0],
                                 likeright1 = child1.likelihood[1],
                                 likeleft0 = child2.likelihood[0],
                                 likeleft1 = child2.likelihood[1],
                                 tR = child1.dist,
                                 tL = child2.dist,
                                 alpha = alpha,
                                 beta = beta)
            node.likelihood = likedict

In [14]:
mytree = toytree.rtree.unittree(ntips = 12)
mytree.draw(tree_style="p")

(<toyplot.canvas.Canvas at 0x7f99fb1cbdc0>,
 <toytree.Render.ToytreeMark at 0x7f99fa1402b0>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="265.0px" viewBox="0 0 350.0 265.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tedb4370e0b37471f85e3fce4527cb245"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 r0 r1 r2 r3 r4 r5 r6 r7 r8 r9 r10 r11 0.0 0.5 1.0

In [20]:
mydata = [0,1,1,1,0,0,1,1,1,0,1,0]
mydict = datatodict(mydata)
mydict

{0: {0: 1, 1: 0},
 1: {0: 0, 1: 1},
 2: {0: 0, 1: 1},
 3: {0: 0, 1: 1},
 4: {0: 1, 1: 0},
 5: {0: 1, 1: 0},
 6: {0: 0, 1: 1},
 7: {0: 0, 1: 1},
 8: {0: 0, 1: 1},
 9: {0: 1, 1: 0},
 10: {0: 0, 1: 1},
 11: {0: 1, 1: 0}}

In [48]:
mytree.set_node_values(feature = "likelihood", values = {0:{0:1,1:0}, 1:{0:0,1:1}})
mytree.get_node_values(feature = "likelihood", show_tips=True)

array(['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', ''], dtype='<U1')

In [38]:
mytree3 = toytree.rtree.unittree(ntips = 3)
mytree3.draw(tree_style="p")

(<toyplot.canvas.Canvas at 0x7f99fb1cbac0>,
 <toytree.Render.ToytreeMark at 0x7f99faa9ed90>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="260.0px" viewBox="0 0 350.0 260.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tc5b2221982ff48b0b1d7a728da094320"> 0 1 2 3 4 r0 r1 r2 0.0 0.5 1.0

In [57]:
adict = {0:2, 1:3, 2:4}
mytree3.set_node_values(feature="feature", values=adict)
mytree3.get_node_values("feature", True, True)

array(['', '', '', '', ''], dtype='<U1')

In [58]:
def full_lik2(x0, likeleft0, likeleft1, likeright0, likeright1, tL, tR, anca):
    """
    Get the likelihood of the data given the observed
    tip states (a, b) brlens (ta, tb) and ancestral
    state (anca) using two rate parameters (x0 array).
    """
    # get likelihood of ancestral states 0 or 1
    condlik = cond_lik2(a, b, ta, tb, x0[0], x0[1])
    
    # get full likelihoodlinke
    lik = (1 - anca) * node.likelihood[0] + (anca) * node.likelihood[1]
    
    # I don't understand this part
    if anca in [0., 1.]:
        lik /= 2
    
    return -lik #np.log(lik)

In [ ]:
def model_fit(a, b, ta, tb, ancestral):
    """
    Find the maximum likelihood estimate of the two
    rate model parameters given the data.
    """
    args = (a, b, ta, tb, ancestral)
    
    # ML estimate
    estimate = minimize(
        fun=full_lik2, 
        x0=np.array([1., 1.]),
        args=args,
        method='L-BFGS-B',
        bounds=((0, 10), (0, 10))
    )
    
    score = -1 * full_lik2(estimate.x, *args)
    result = {
        "alpha": round(estimate.x[0], 3),
        "beta": round(estimate.x[1], 3), 
        "lik": round(score, 3),
        "convergence": estimate.success,
    }
    return result
    